In [21]:
import geopandas as gp
from shapely import Point, LineString
from tqdm.notebook import tqdm
import numpy as np
tqdm.pandas()

In [22]:
raw_full_trip_gdf = gp.read_file("../data/freemove/freemove_smooth.geojson", geometry='geometry')

In [23]:
# get start and end points (these steps will be done in data loader so we just have to replicate them here in order to get same data format)

# Create SP and EP columns
raw_full_trip_gdf['TRIP_SP'] = raw_full_trip_gdf.geometry.apply(lambda x: Point(x.coords[0]))
raw_full_trip_gdf['TRIP_EP'] = raw_full_trip_gdf.geometry.apply(lambda x: Point(x.coords[-1]))

In [24]:
def cut_epz(row, rad_l=100, rad_h=300):
    """This function takes in a row of a GeoDataframe and returns the private linestring truncated with SP and EP privacy zones of a random radius between the specified parameters in meters.

    Args:
        row (_type_): _description_

    Returns:
        _type_: _description_
    """
    # Generate circular privacy zone around start and end point with random radius uniform between 100-300m
    sp_rad = np.random.randint(rad_l,rad_h+1)
    ep_rad = np.random.randint(rad_l,rad_h+1)


    sp_zone = row.TRIP_SP.buffer(sp_rad)
    ep_zone = row.TRIP_EP.buffer(ep_rad)

    # Remove points of linestring that lie in either privacy zone and return empty linestring if entire trajectroy has been truncated
    try:
        priv_linestring = LineString([x for x in row.geometry.coords if not Point(x).within(sp_zone) and not Point(x).within(ep_zone)])
        if len(priv_linestring.coords) < 1:
            return LineString()
        else:
            return priv_linestring
    except Exception as e:
        return LineString()
    
    

In [25]:
raw_full_trip_gdf['geometry'] = raw_full_trip_gdf.progress_apply(lambda x: cut_epz(x), axis = 1)

raw_full_trip_gdf.head(20)

  0%|          | 0/1408 [00:00<?, ?it/s]

,traj_id,start_t,end_t,length,direction,user_id,geometry,TRIP_SP,TRIP_EP
0,978933,2022-10-21T14:59:24,2022-10-21T15:25:01,6370.874696,72.645427,17246,"LINESTRING (4546258.977 3274275.786, 4546263.9...",POINT (4546050.591 3274210.566),POINT (4549927.602 3275422.173)
1,978934,2022-10-21T16:07:38,2022-10-21T16:43:11,396.033850,78.138936,17246,LINESTRING EMPTY,POINT (4549919.181 3275243.186),POINT (4549967.537 3275253.342)
2,978946,2022-10-21T19:16:16,2022-10-21T19:32:46,4679.385964,255.753824,17246,"LINESTRING (4549694.337 3275405.887, 4549687.3...",POINT (4549932.284 3275234.547),POINT (4546038.845 3274246.016)
3,978955,2022-10-21T20:41:59,2022-10-21T21:29:28,13123.732328,112.545524,17246,"LINESTRING (4546306.609 3274312.387, 4546308.2...",POINT (4546158.835 3274262.968),POINT (4553816.697 3271083.847)
4,980322,2022-10-23T09:32:44,2022-10-23T10:01:43,2060.403029,32.198363,16947,"LINESTRING (4550106.051 3274249.028, 4550105.3...",POINT (4550247.794 3274092.506),POINT (4550266.007 3274121.429)
5,985011,2022-10-31T00:34:27,2022-10-31T00:48:44,4349.848395,106.017690,16667,"LINESTRING (4547233.512 3265693.496, 4547237.8...",POINT (4547140.367 3265745.160),POINT (4550935.141 3264655.758)
6,985073,2022-10-31T06:49:25,2022-10-31T07:27:35,14167.779227,273.898113,16646,"LINESTRING (4557480.822 3273780.020, 4557480.6...",POINT (4557611.520 3273566.340),POINT (4545866.836 3274366.623)
7,985091,2022-10-31T07:36:55,2022-10-31T08:02:12,6284.222628,247.383788,17267,"LINESTRING (4546676.120 3273222.094, 4546670.3...",POINT (4546897.099 3273387.961),POINT (4542678.531 3271630.540)
8,985112,2022-10-31T08:03:55,2022-10-31T08:32:29,10432.192451,179.448228,16361,"LINESTRING (4543933.580 3274688.597, 4543933.4...",POINT (4543861.585 3274581.065),POINT (4543952.623 3265127.937)
9,985118,2022-10-31T08:39:14,2022-10-31T08:53:23,3100.711327,343.764337,16354,"LINESTRING (4546643.579 3269869.190, 4546643.7...",POINT (4546780.605 3269684.946),POINT (4546056.356 3272172.046)


In [26]:
# drop helper columns
raw_full_trip_gdf.drop(['TRIP_SP', 'TRIP_EP'], axis=1, inplace=True)

# Create geojson
raw_full_trip_gdf.to_file("../data/freemove/freemove_smooth_private.geojson", driver='GeoJSON')

/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
